### So this note book as to be run sequentially because of the dropping of indexes and not updating them again

In [1]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(20000)

Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (20000 rows)...
Finished inserting 20000 rows into 'products' in 0.35s.


This is just to test out how mysql will explain different joins and if index'es affect them

In [2]:
display(show_table_indexes("products", get_database_name()))
display(show_table_indexes("categories", get_database_name()))

,Index Name,Columns,Type,Non Unique
0,availability_id,availability_id,BTREE,True
1,brand_id,brand_id,BTREE,True
2,category_id,category_id,BTREE,True
3,color_id,color_id,BTREE,True
4,PRIMARY,id,BTREE,False


,Index Name,Columns,Type,Non Unique
0,name,name,BTREE,False
1,PRIMARY,id,BTREE,False


In [ ]:
query = """
SELECT * FROM products INNER JOIN categories ON products.category_id = categories.id
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 22 rows fetched, 1.12 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,"category_id,idx_test",idx_test,5,const,22,100.0,Using index condition; Using where
1,1,SIMPLE,categories,None,eq_ref,PRIMARY,PRIMARY,4,test_database.products.category_id,1,100.0,None


In [4]:
query = """
SELECT * FROM products LEFT JOIN categories ON products.category_id = categories.id
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 20000 rows fetched, 97.86 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,19832,100.0,None
1,1,SIMPLE,categories,None,eq_ref,PRIMARY,PRIMARY,4,test_database.products.category_id,1,100.0,None


In [5]:
query = """
SELECT * FROM products RIGHT JOIN categories ON products.category_id = categories.id
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 20000 rows fetched, 97.20 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,categories,None,index,None,name,1023,None,34,100.0,Using index
1,1,SIMPLE,products,None,ref,category_id,category_id,5,test_database.categories.id,583,100.0,None


In [6]:
drop_non_clustered_indexes("categories", get_database_name())
display(show_table_indexes("products", get_database_name()))
display(show_table_indexes("categories", get_database_name()))


,Index Name,Columns,Type,Non Unique
0,availability_id,availability_id,BTREE,True
1,brand_id,brand_id,BTREE,True
2,category_id,category_id,BTREE,True
3,color_id,color_id,BTREE,True
4,PRIMARY,id,BTREE,False


,Index Name,Columns,Type,Non Unique
0,PRIMARY,id,BTREE,False


In [10]:
query = """
SELECT * FROM products INNER JOIN categories ON products.category_id = categories.id
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 20000 rows fetched, 116.39 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,categories,None,ALL,PRIMARY,None,None,None,34,100.0,None
1,1,SIMPLE,products,None,ref,category_id,category_id,5,test_database.categories.id,583,100.0,None


In [8]:
query = """
SELECT * FROM products LEFT JOIN categories ON products.category_id = categories.id
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 20000 rows fetched, 97.67 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,19832,100.0,None
1,1,SIMPLE,categories,None,eq_ref,PRIMARY,PRIMARY,4,test_database.products.category_id,1,100.0,None


In [9]:
query = """
SELECT * FROM products RIGHT JOIN categories ON products.category_id = categories.id
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 20000 rows fetched, 96.36 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,categories,None,ALL,None,None,None,None,34,100.0,None
1,1,SIMPLE,products,None,ref,category_id,category_id,5,test_database.categories.id,583,100.0,None
